# Project - Fashion Customer Service Assistant

We'll now bring together what we've learned to make an AI Fashion Customer Service assistant for fashion industry.

In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful and professional fashion customer service assistant."
system_message += "Provide clear, courteous responses in no more than 1 sentence."
system_message += "Always ensure your answers are accurate and relevant."
system_message += "If you are unsure about something, politely state that you don't have the information."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
# Let's start by making a useful function

def fashion_customer_service_assistant(user_query):
    query = user_query.strip().lower()

    if "price" in query or "how much" in query:
        if "jeans" in query:
            return "Our jeans start at $49."
        elif "jacket" in query:
            return "Our jackets range from $89 to $199."
        elif "shoes" in query:
            return "Shoes are priced between $59 and $129."
        else:
            return "I'm happy to help with pricing — could you specify the item?"

    elif "return" in query or "refund" in query:
        return "You can return items within 30 days for a full refund."

    elif "delivery" in query or "shipping" in query:
        return "Standard delivery takes 3–5 business days."

    elif "size" in query or "fit" in query:
        return "Our size guide is available on each product page to help you choose the right fit."

    elif "customer support" in query or "contact" in query:
        return "Our customer support team is available Monday to Saturday, 9am to 6pm."

    else:
        return "I'm not sure about that, but I’m here to help with product, order, or return-related questions."

In [6]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_item_price",
    "description": (
        "Get the price of a fashion item. "
        "Call this whenever a customer asks for the price of an item, "
        "for example when they say 'How much is this dress?' or 'What does a jacket cost?'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "item_name": {
                "type": "string",
                "description": "The name or type of the fashion item the customer is asking about (e.g., dress, shoes, jacket)",
            },
        },
        "required": ["item_name"],
        "additionalProperties": False
    }
}

In [7]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [9]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    item = arguments.get('item_name')
    price = get_item_price(item)
    response = {
        "role": "tool",
        "content": json.dumps({"item_name": item, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response, item

In [11]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7895
* Running on public URL: https://f9a7cccb7429024d25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
